In [42]:
import numpy as np

import tensorflow as tf

from data_processing import *
import keras
import keras.backend as K

from keras.optimizers import Adam,SGD
from keras.callbacks import TensorBoard, ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

from model import ResNet50

def _main():
    image_path = 'C:/Users/emage/OneDrive/Desktop/apple_k/data/train_image/'
    labels_path = 'C:/Users/emage/OneDrive/Desktop/apple_k/data/train.csv'
    csv = pd.read_csv(labels_path)
    image_named = np.array(csv['image_id'])
    y = csv.loc[:, 'healthy':].values
    
    train ,val, y_train,y_val = train_test_split(image_named, y, test_size = 0.22, random_state = 42, shuffle = True)

    freeze = 1

    # step = 607
    num_train = 1420
    num_val = 401
    
    log_dir = 'log/imageNet/letter_image_Relu_only_Dense/'
    check_path(log_dir)
    weights_path = 'imageNet'

    model = ResNet50(weights = 'imageNet')
    
    
    # # temporary use Adam and
    adam = Adam(lr = 0.001)
    # logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, mode = 'min')
   
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr = 1e-6)


    # Freeze FC layers
    for i in range((len(model.layers)-freeze)):
        model.layers[i].trainable = False

    # model.compile(optimizer = adam, 
    #             loss = 'categorical_crossentropy',
    #             # loss = [categorical_focal_loss(alpha=.25, gamma=2)],
    #             metrics = ['accuracy'])
    model.compile(optimizer = adam, 
            loss = 'categorical_crossentropy',
            metrics = ['accuracy'])

    batch_size  = 10
    
    history = model.fit_generator(train_generator(image_path, train, batch_size, y_train, num_train),
        epochs = 10, 
        steps_per_epoch = num_train//batch_size, 
        validation_data = val_generator(image_path, val, batch_size, y_val, num_val),
        validation_steps = num_val//batch_size,
        initial_epoch = 0,
        verbose = 1,
        callbacks=[checkpoint])
    model.save_weights(log_dir+'model.h5')
    import pickle
    with open('history', 'wb') as f:
        pickle.dump(history.history, f)


def check_path(path):
    os.makedirs(path) if not os.path.exists(path) else None

if __name__ == "__main__":
    h = _main()

Epoch 1/10
142/142 [==============================] - 29s 201ms/step - loss: 1.3713 - accuracy: 0.4225 - val_loss: 1.5475 - val_accuracy: 0.3325
Epoch 2/10
142/142 [==============================] - 27s 194ms/step - loss: 1.0832 - accuracy: 0.5712 - val_loss: 1.2794 - val_accuracy: 0.3375
Epoch 3/10
142/142 [==============================] - 31s 222ms/step - loss: 1.0283 - accuracy: 0.5884 - val_loss: 1.4235 - val_accuracy: 0.0925
Epoch 4/10
142/142 [==============================] - 32s 227ms/step - loss: 0.9819 - accuracy: 0.6135 - val_loss: 1.2046 - val_accuracy: 0.2750
Epoch 5/10
142/142 [==============================] - 31s 222ms/step - loss: 0.8892 - accuracy: 0.6448 - val_loss: 1.3675 - val_accuracy: 0.2325
Epoch 6/10
142/142 [==============================] - 32s 228ms/step - loss: 0.8963 - accuracy: 0.6635 - val_loss: 1.3041 - val_accuracy: 0.2775
Epoch 7/10
142/142 [==============================] - 32s 229ms/step - loss: 0.9957 - accuracy: 0.6307 - val_loss: 1.3622 - val_ac

In [23]:
import json
print(h.history)
print(type(h.history['accuracy'][0]))
# with open('history.json', 'w') as f:
#     json.dump(h.history,f)
import pickle
with open('history', 'wb') as f:
    pickle.dump(h.history, f)

{'val_loss': [2.093095541000366], 'val_accuracy': [0.2800000011920929], 'loss': [1.2988779242609587], 'accuracy': [0.45774648]}
<class 'numpy.float32'>


In [43]:

a = pickle.load(open('history','rb'))

In [44]:
a

{'val_loss': [1.5474756956100464,
  1.2793667316436768,
  1.423488736152649,
  1.2045801877975464,
  1.3674927949905396,
  1.3040878772735596,
  1.3621649742126465,
  1.2194452285766602,
  1.2119163274765015,
  1.3183153867721558],
 'val_accuracy': [0.33250001072883606,
  0.3375000059604645,
  0.0925000011920929,
  0.2750000059604645,
  0.23250000178813934,
  0.2775000035762787,
  0.27250000834465027,
  0.2775000035762787,
  0.2800000011920929,
  0.2775000035762787],
 'loss': [1.371321764630331,
  1.083157848829115,
  1.0283325197516193,
  0.9819496081433665,
  0.8892142077461934,
  0.896329088267726,
  0.9956779001464306,
  0.8438075865343423,
  0.8826559870700601,
  0.8468351330818005],
 'accuracy': [0.4225352,
  0.571205,
  0.5884194,
  0.6134585,
  0.64475745,
  0.6635368,
  0.63067293,
  0.68075114,
  0.65258217,
  0.685446]}